In [1]:
pip install transformers

In [2]:
from transformers import T5ForConditionalGeneration, T5TokenizerFast
hfmodel = T5ForConditionalGeneration.from_pretrained("ThomasSimonini/t5-end2end-question-generation")
checkpoint = "t5-base"
model = T5ForConditionalGeneration.from_pretrained(checkpoint)
tokenizer = T5TokenizerFast.from_pretrained(checkpoint)
tokenizer.sep_token = '<sep>'
tokenizer.add_tokens(['<sep>'])
model.resize_token_embeddings(len(tokenizer))
# Check the sep_token_id to verify that it was added to the tokenizer
tokenizer.sep_token_id

def hf_run_model(input_string, **generator_args):
  generator_args = {
  "max_length": 256,
  "num_beams": 4,
  "length_penalty": 1.5,
  "no_repeat_ngram_size": 3,
  "early_stopping": True,
  }
  input_string = "generate questions: " + input_string + " </s>"
  input_ids = tokenizer.encode(input_string, return_tensors="pt")
  res = hfmodel.generate(input_ids, **generator_args)
  output = tokenizer.batch_decode(res, skip_special_tokens=True)
  output = [item.split("<sep>") for item in output]
  return output

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [3]:

text = "Forrest Gump is a 1994 American comedy-drama film directed by Robert Zemeckis and written by Eric Roth. \
It is based on the 1986 novel of the same name by Winston Groom and stars Tom Hanks, Robin Wright, Gary Sinise, \
Mykelti Williamson and Sally Field. The story depicts several decades in the life of Forrest Gump (Hanks), \
a slow-witted but kind-hearted man from Alabama who witnesses and unwittingly influences several defining \
historical events in the 20th century United States. The film differs substantially from the novel."

hf_run_model(text)

#output
#[['Who directed the 1994 film Forrest Gump?',
#  ' Who wrote the 1994 movie?',
#  ' What is the film based on?',
#  ' Which movie stars Tom Hanks, Robin Wright and Gary Sinise?',
#  '']]


[['Who directed the 1994 film Forrest Gump?',
  ' Who wrote the 1994 movie?',
  ' What is the film based on?',
  ' Which movie stars Tom Hanks, Robin Wright and Gary Sinise?',
  '']]

In [4]:
text_passage = "A vector in this context is a mathematical representation of an object or data point, \
where each element of the vector corresponds to a specific feature or attribute of the object. \
For example, in an image recognition system, a vector could represent an image, with each element of the \
vector representing a pixel value or a descriptor/characteristic of that pixel. In a music recommendation system, \
each vector would represent a song, and each element of the vector would represent a characteristic song such as tempo,\
 genre, lyrics, and so on. Vector databases are optimized for **storing** and **querying** these high-dimensional vectors efficiently, \
 and they often using specialized data structures and indexing techniques such as Hierarchical Navigable Small World (HNSW) -- \
 which is used to implement Approximate Nearest Neighbors -- and Product Quantization, among others. \
 These databases enable fast similarity and semantic search while allowing users to find vectors that are the closest to a given\
  query vector based on some distance metric. The most commonly used distance metrics are Euclidean Distance, \
  Cosine Similarity, and Dot Product, and these three are fully supported Qdrant."


hf_run_model(text_passage)

[['What is a mathematical representation of an object or data point?',
  ' In an image recognition system, a vector could represent an image, with each element of the vector representing a pixel value or a descriptor/characteristic of what pixel?',
  ' What is HNSW used to implement?',
  '']]

In [7]:
text_vdb = "## What Are Vector Databases? ![dbs](https://raw.githubusercontent.com/ramonpzg/mlops-sydney-2023/main/images/databases.png) \
 Vector databases are a type of database designed to store and query high-dimensional vectors efficiently. \
 In traditional [OLTP](https://www.ibm.com/topics/oltp) and [OLAP](https://www.ibm.com/topics/olap) \
 databases (as seen in the image above), data is organized in rows and columns (and these are called **Tables**), \
 and queries are performed based on the values in those columns. However, in certain applications including image recognition,\
  natural language processing, and recommendation systems, data is often represented as vectors in a high-dimensional space, and these vectors,\
   plus an id and a payload, are the elements we store in something called a **Collection** a vector database like Qdrant."

text_vdb1 = "What is Qdrant? [Qdrant](http://qdrant.tech) is a vector similarity search engine that provides a \
production-ready service with a convenient API to store, search, and manage points (i.e. vectors) with an additional payload.\
 You can think of the payloads as additional pieces of information that can help you hone in on your search and also receive \
 useful information that you can give to your users. You can get started using Qdrant with the Python `qdrant-client`, \
 by pulling the latest docker image of `qdrant` and connecting to it locally, or by trying out [Qdrant's Cloud](https://cloud.qdrant.io/) \
 free tier option until you are ready to make the full switch. With that out of the way, let's talk about what are vector databases."

#hf_run_model(text_vdb)

#hf_run_model(text_vdb1)

In [ ]:
!pip install openai

In [14]:
import openai
from getpass import getpass
import os

if not (OPENAI_API_KEY := os.getenv("OPENAI_API_KEY")):
    OPENAI_API_KEY = getpass("🔑 Enter your OpenAI API key: ")
openai.api_key = OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

model ="gpt-3.5-turbo"
client = OpenAI()

 # Function to generate synthetic question-answer pairs
def generate_question_answer(context):

  prompt = f"""Generate a question and answer pair based keeping in mind the following:
  Please generate a clear and concise question that requires understanding of the content provided in the document chunk. Ensure that the question is specific, relevant, and not too broad.
  Avoid questions such as 'in the given passage or document chunk' kind of questions. Ensure the question is about the concept the document chunk is about.
  Provide a complete , detailed and accurate answer to the question. Make sure that the answer addresses the question directly and comprehensively, drawing from the information provided in the document chunk.
  Use technical terminology appropriately and maintain clarity throughout the response.
  Based on this Context : {context}
  """

  response = client.chat.completions.create(
        model = model,
        messages = [
        {'role': 'user', 'content': prompt}
          ],
        temperature=0.5,
        max_tokens=200,
        n=1,
        stop=None
    )
  return response.choices[0].message.content


In [15]:
# Generate question-answer pairs for each chunk


context = "What is Qdrant? [Qdrant](http://qdrant.tech) is a vector similarity search engine that provides a \
production-ready service with a convenient API to store, search, and manage points (i.e. vectors) with an additional payload.\
You can think of the payloads as additional pieces of information that can help you hone in on your search and also receive \
useful information that you can give to your users. You can get started using Qdrant with the Python `qdrant-client`, \
by pulling the latest docker image of `qdrant` and connecting to it locally, or by trying out [Qdrant's Cloud](https://cloud.qdrant.io/) \
free tier option until you are ready to make the full switch. With that out of the way, let's talk about what are vector databases."

question_answer_pair = generate_question_answer(context)
print(question_answer_pair)

What is Qdrant and how can it be used in vector similarity search?

Qdrant is a vector similarity search engine that offers a production-ready service with an API for storing, searching, and managing points (vectors) along with additional payload information. The payloads can provide extra details to refine searches and offer valuable information to users. Qdrant can be utilized through the Python `qdrant-client`, by downloading the latest docker image of `qdrant` and connecting to it locally, or by exploring the free tier option of [Qdrant's Cloud](https://cloud.qdrant.io/) before transitioning to the full version.


In [11]:
# Generate question-answer pairs for each chunk
question_answer_pair = generate_question_answer(text_vdb1)
print(question_answer_pair)

1. What are vector databases and how are they related to Qdrant?
   Vector databases are databases that store and manage data points represented as vectors in a multi-dimensional space. Qdrant is a vector similarity search engine that provides a production-ready service with a convenient API to store, search, and manage points (vectors) with an additional payload. In essence, Qdrant is a specialized type of vector database that focuses on efficient similarity search operations on vectors.

2. How can users get started using Qdrant and what options are available to them?
   Users can get started using Qdrant by using the Python `qdrant-client`, pulling the latest docker image of `qdrant` and connecting to it locally, or trying out Qdrant's Cloud free tier option until they are ready to make the full switch. These options provide users with different ways to access and utilize Qdrant based on their specific needs and preferences.
